In [1]:
import os

from dotenv import load_dotenv

from genai_evaluator.clients import OpenAILLMClient, TemplateStore

load_dotenv()

True

In [2]:
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [3]:
llm_client = OpenAILLMClient(
    key=AZURE_OPENAI_API_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
)

In [4]:
from genai_evaluator.clients.data_clients import TemplateStore

template_store = TemplateStore(dir_path="genai_evaluator/templates", do_validation=True)

Loading templates from genai_evaluator\templates


In [5]:
from genai_evaluator.flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_ret)

Predicted statements: ["France's capital city is Paris.", 'Paris is known for the Eiffel Tower.']
Ground truth statements: ['The capital of France is Paris.']
{"verdicts":[{"verdict":"yes","reason":null},{"verdict":"no","reason":"The predicted statement mentions Paris being known for the Eiffel Tower, which is not addressed in the ground truth statements."}]}
Predicted statements: ['The capital of France is Paris.']
Ground truth statements: ["France's capital city is Paris.", 'Paris is known for the Eiffel Tower.']
{"verdicts":[{"verdict":"yes","reason":null}]}
{'relevancy': 1.0}


In [6]:
import os

from promptflow.tracing import start_trace

from genai_evaluator.flows.generation_eval_flow import generation_eval_flow

os.environ["PF_DISABLE_TRACING"] = "FALSE"
start_trace(collection="trace_rag_metrics")

dict_eval_gen = generation_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_gen)

Prompt flow service has started...
Predicted statements: ["France's capital city is Paris.", 'Paris is known for the Eiffel Tower.']
Ground truth statements: ['The capital of France is Paris.']
{"verdicts":[{"verdict":"yes","reason":null},{"verdict":"idk","reason":null}]}
Predicted statements: ['The capital of France is Paris.']
Ground truth statements: ["France's capital city is Paris.", 'Paris is known for the Eiffel Tower.']
{"verdicts":[{"verdict":"yes","reason":null}]}
{'faithfulness': 1.0, 'precision': 1.0, 'recall': 1.0}


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xb1f3c0f819371ab685daae42ffda44c7
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x51f7ad817c97243fa192a13c4572c7cf
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xbfd137682e17af6b94de1d991f22a633
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xeaee367a6ad5f04f91ddc5fa448d5025
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xc347b02249620f66a7668d654016c640
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x1e4f09d1f84917da3edfca080

ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\UR616WJ\genai_evaluator\.venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\UR616WJ\genai_evaluator\.venv\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\UR616WJ\AppData\Roaming\uv\python\cpython-3.13.1-windows-x86_64-none\Lib\http\client.py", line 1428, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "C:\Users\UR616WJ\AppData\Roaming\uv\python\cpython-3.13.1-windows-x86_64-none\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "C:\Users\UR616WJ\AppData\Roaming\uv\python\cpython-3.13.1-windows-x86_64-none\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x759cab7e817900bae40499bf95c5b1dc
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0xe35c6ab2f3e72e986d47b49afe0cc1f5
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x2fdec7488fb0d23d4f81c7e077ffa79b
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x2f34840a68a89371e9b3ce0c93ce67a0
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x8f296f65fadaa078f2c811c8a4a292f5
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=trace_rag_metrics&uiTraceId=0x4a00ee96f96c589d690e8197e

In [7]:
from genai_evaluator.flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_ret)

Predicted statements: ['Italian pesto is usually made with spinach.', 'Italian pesto is usually made with almonds.', 'Italian pesto is usually made with cheddar cheese.', 'Italian pesto is usually made with olive oil.']
Ground truth statements: ['Traditional Italian pesto is made with basil.', 'Traditional Italian pesto is made with garlic.', 'Traditional Italian pesto is made with pine nuts.', 'Traditional Italian pesto is made with Parmesan cheese.', 'Traditional Italian pesto is made with olive oil.']
{"verdicts":[{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with basil, not spinach."},{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with pine nuts, not almonds."},{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with Parmesan cheese, not cheddar cheese."},{"verdict":"yes","reason":null}]}
Predicted statements: ['Traditional Italian pesto is made with basil.', 'T

In [ ]:
import os

from promptflow.tracing import start_trace

os.environ["PF_DISABLE_TRACING"] = "FALSE"


start_trace()
dict_eval_gen = generation_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_gen)

Prompt flow service has started...
{"verdicts":[{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with basil, not spinach."},{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with pine nuts, not almonds."},{"verdict":"no","reason":"The ground truth states that traditional Italian pesto is made with Parmesan cheese, not cheddar cheese."},{"verdict":"yes","reason":null}]}
{"verdicts":[{"verdict":"no","reason":"The ground truth statements do not mention basil as an ingredient for Italian pesto."},{"verdict":"no","reason":"The ground truth statements do not mention garlic as an ingredient for Italian pesto."},{"verdict":"no","reason":"The ground truth statements do not mention pine nuts as an ingredient for Italian pesto."},{"verdict":"no","reason":"The ground truth statements do not mention Parmesan cheese as an ingredient for Italian pesto."},{"verdict":"yes","reason":null}]}
{'faithfulness': 0.25, 'precision':

RAG evaluation flow

In [9]:
from genai_evaluator.clients.embedding_client import OpenAIEmbeddingClient
from genai_evaluator.clients.vector_store_client import FAISSClient
from genai_evaluator.flows.rag_flow import rag_flow

embedding_client = OpenAIEmbeddingClient(
    credential=AZURE_OPENAI_API_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    model="text-embedding-ada-002",
)
vector_store = FAISSClient(embedding_client=embedding_client)

In [10]:

from pathlib import Path

context = Path("../data/equality-and-diversity-policy.pdf")
batch_size =  400
# context = [
#     "To build muscle effectively, focus on compound exercises such as squats, deadlifts, bench press, and pull-ups.",
#     "Progressive overload is essential — gradually increasing the weight or reps over time will stimulate muscle growth.",
#     "Eating a calorie surplus with adequate protein intake (1.6-2.2g per kg of body weight) supports muscle hypertrophy.",
#     "Rest and recovery are crucial; muscles grow during rest periods, so aim for at least 7-8 hours of sleep per night.",
#     "Train each major muscle group at least twice a week with a combination of volume and intensity.",
#     "A typical beginner mass-building routine includes 3–4 full-body workouts per week focusing on strength and hypertrophy.",
#     "Supplements like creatine monohydrate and whey protein can support your muscle-building goals when used correctly.",
# ]
vector_store.ingest_documents(context, batch_size=batch_size)

2025-06-10 16:19:21.515 | INFO     | genai_evaluator.clients.vector_store_client:ingest_documents:102 - Total documents ingested: 17012


In [11]:
question = "What are the best exercises for building muscle?"
answer_gt = "The best exercises for building muscle include compound movements like squats, deadlifts, bench press, and pull-ups."

In [12]:
from promptflow.tracing import start_trace

os.environ["PF_DISABLE_TRACING"] = "FALSE"
start_trace(collection="trace_rag_metrics")

result = rag_flow(
    question=question,
    answer_gt=answer_gt,
    llm_client=llm_client,
    template_store=template_store,
    vector_store=vector_store,
    num_results=3,
    system_prompt="Answer the question based on the provided context.",
)
print(result)

Prompt flow service has started...


Predicted statements: ['Building muscle effectively involves a combination of compound and isolation exercises that target major muscle groups.', 'Compound exercises engage multiple muscle groups and joints, making them highly effective for building overall strength and mass.', 'Examples of compound exercises include squats, deadlifts, bench press, pull-ups/chin-ups, overhead press, and barbell rows.', 'Isolation exercises focus on specific muscles, helping to refine and build definition.', 'Examples of isolation exercises include bicep curls, tricep extensions, lateral raises, leg curls, and calf raises.', 'Core exercises support overall muscle development and stability.', 'Examples of core exercises include planks, hanging leg raises, and Russian twists.', 'Tips for muscle growth include progressive overload, proper form, rest and recovery, nutrition, and consistency.', 'Incorporating these exercises into a well-rounded program will help build muscle effectively.']
Ground truth state

{"verdicts":[{"verdict":"yes","reason":null},{"verdict":"yes","reason":null}]}
{'relevancy': 1.0, 'faithfulness': None, 'precision': 1.0, 'recall': 1.0}
